In [83]:
import tensorflow as tf
import numpy as np

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="C:/Users/AI/Desktop/Tensorflow/custom_model_lite/detect.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input shape:", input_details[0]['shape'])
print("Input type:", input_details[0]['dtype'])
print("Output shape:", output_details[0]['shape'])
print("Output type:", output_details[0]['dtype'])


# Get details of all layers
for i, layer in enumerate(interpreter.get_tensor_details()):
    print(f"Layer {i}: {layer['name']} - {layer['shape']}")



# Generate random input data as an example
input_data = np.random.rand(*input_details[0]['shape']).astype(np.float32)

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Print the output
print("Output:", output_data)


Input shape: [  1 320 320   3]
Input type: <class 'numpy.float32'>
Output shape: [ 1 10]
Output type: <class 'numpy.float32'>
Layer 0: serving_default_input:0 - [  1 320 320   3]
Layer 1: WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead/Reshape/shape - [3]
Layer 2: WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead/Reshape_1/shape - [3]
Layer 3: anchors - [12804     4]
Layer 4: ssd_mobile_net_v2_fpn_keras_feature_extractor/FeatureMaps/top_down/nearest_neighbor_upsampling/nearest_neighbor_upsampling/Reshape/shape - [4]
Layer 5: ssd_mobile_net_v2_fpn_keras_feature_extractor/FeatureMaps/top_down/nearest_neighbor_upsampling/nearest_neighbor_upsampling/Reshape_1/shape - [4]
Layer 6: ssd_mobile_net_v2_fpn_keras_feature_extractor/FeatureMaps/top_down/nearest_neighbor_upsampling/nearest_neighbor_upsampling_1/Reshape/shape - [4]
Layer 7: ssd_mobile_net_v2_fpn_keras_feature_extractor/FeatureMaps/top_down/nearest_neighbor_upsampling/nearest_neighbor_ups

In [81]:
import tensorflow as tf
from decimal import Decimal
from tabulate import tabulate

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="C:/Users/AI/Desktop/Tensorflow/custom_model_lite/detect.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Initialize table data
table_data = []

# Get details of all layers
total_params = Decimal(0)  # Use Decimal for total_params to handle larger values
total_flops = Decimal(0)  # Use Decimal for total_flops to handle larger values

for i, layer in enumerate(interpreter.get_tensor_details()):
    layer_name = layer['name']
    layer_output_shape = layer['shape']

    if len(layer_output_shape) > 0:  # Check if the layer has output shape
        layer_type = None
        
        # Check for specific keywords in the layer name to determine the type
        if 'conv' in layer_name.lower():
            layer_type = 'Convolutional Layer'
        elif 'dense' in layer_name.lower() or 'fullyconnected' in layer_name.lower():
            layer_type = 'Fully Connected Layer'
        # Add more conditions for other types of layers as needed

        if layer_type:
            # Count parameters and FLOPs for convolutional layers
            if 'conv2d' in layer_name.lower():
                if len(layer_output_shape) >= 4:  # Check if the shape has enough dimensions
                    filter_shape = layer_output_shape[1:3]
                    in_channels = layer_output_shape[3]
                    out_channels = layer_output_shape[0]
                    kernel_size = int(filter_shape[0]) * int(filter_shape[1])

                    # Convert numpy integers to regular integers before conversion to Decimal
                    params = Decimal(int(kernel_size * in_channels) * int(out_channels) + int(out_channels))  # Weight + bias
                    flops = Decimal(int(kernel_size * in_channels) * int(out_channels) * int(layer_output_shape[1]) * int(layer_output_shape[2]))  # MACs

                    total_params += params
                    total_flops += flops
                    
                    table_data.append([layer_type, layer_output_shape, params, flops])
                else:
                    table_data.append([layer_type, layer_output_shape, "N/A", "N/A"])
            else:
                # Assuming other layers (e.g., fully connected) have only bias parameters
                params = Decimal(int(layer_output_shape[0]) + 1)  # Bias only
                flops = "N/A"  # FLOPs calculation not possible without more info

                total_params += params

                table_data.append([layer_type, layer_output_shape, params, flops])
        else:
            table_data.append(["Unknown Layer Type", layer_output_shape, "N/A", "N/A"])
    else:
        table_data.append(["Unknown Layer Type", layer_name, "N/A", "N/A"])

# Print the table without warnings
headers = ["Layer Type", "Output Shape", "Parameters", "FLOPs"]
print(tabulate(table_data, headers=headers, tablefmt="plain"))
print(f"Total Parameters: {total_params}")
print(f"Total FLOPs: {total_flops}")


Layer Type           Output Shape           Parameters    FLOPs
Unknown Layer Type   [  1 320 320   3]      N/A           N/A
Convolutional Layer  [3]                    4             N/A
Convolutional Layer  [3]                    4             N/A
Unknown Layer Type   [12804     4]          N/A           N/A
Unknown Layer Type   [4]                    N/A           N/A
Unknown Layer Type   [4]                    N/A           N/A
Unknown Layer Type   [4]                    N/A           N/A
Unknown Layer Type   [4]                    N/A           N/A
Convolutional Layer  [32]                   33            N/A
Convolutional Layer  [32]                   33            N/A
Unknown Layer Type   [96]                   N/A           N/A
Unknown Layer Type   [96]                   N/A           N/A
Unknown Layer Type   [144]                  N/A           N/A
Unknown Layer Type   [144]                  N/A           N/A
Unknown Layer Type   [144]                  N/A           N/A
Unknow

In [76]:
import tensorflow as tf
from decimal import Decimal
from tabulate import tabulate

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="C:/Users/AI/Desktop/Tensorflow/custom_model_lite/detect.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Initialize table data
table_data = []

# Get details of all layers
total_params = Decimal(0)  # Use Decimal for total_params to handle larger values
total_flops = Decimal(0)  # Use Decimal for total_flops to handle larger values
conv_params = Decimal(0)  # Accumulator for convolutional layer parameters
conv_flops = Decimal(0)  # Accumulator for convolutional layer FLOPs

for i, layer in enumerate(interpreter.get_tensor_details()):
    layer_name = layer['name']
    layer_output_shape = layer['shape']

    if len(layer_output_shape) > 0:  # Check if the layer has output shape
        layer_type = None
        
        # Check for specific keywords in the layer name to determine the type
        if 'conv' in layer_name.lower():
            layer_type = 'Convolutional Layer'
        elif 'dwise' in layer_name.lower() or 'fullyconnected' in layer_name.lower():
            layer_type = 'Fully Connected Layer'
        elif 'pool' in layer_name.lower():
            layer_type = 'Pooling Layer'
        elif 'activation' in layer_name.lower():
            layer_type = 'Activation Layer'
        elif 'dropout' in layer_name.lower():
            layer_type = 'Dropout Layer'
        elif 'flatten' in layer_name.lower():
            layer_type = 'Flatten Layer'
        # Add more conditions for other types of layers as needed

        if layer_type:
            # Count parameters and FLOPs for convolutional layers
            if 'conv2d' in layer_name.lower():
                if len(layer_output_shape) >= 4:  # Check if the shape has enough dimensions
                    filter_shape = layer_output_shape[1:3]
                    in_channels = layer_output_shape[3]
                    out_channels = layer_output_shape[0]
                    kernel_size = int(filter_shape[0]) * int(filter_shape[1])

                    # Convert numpy integers to regular integers before conversion to Decimal
                    params = Decimal(int(kernel_size * in_channels) * int(out_channels) + int(out_channels))  # Weight + bias
                    flops = Decimal(int(kernel_size * in_channels) * int(out_channels) * int(layer_output_shape[1]) * int(layer_output_shape[2]))  # MACs

                    total_params += params
                    total_flops += flops
                    conv_params += params
                    conv_flops += flops
                else:
                    table_data.append([layer_type, layer_output_shape, "N/A", "N/A"])
            else:
                # Assuming other layers (e.g., fully connected) have only bias parameters
                params = Decimal(int(layer_output_shape[0]) + 1)  # Bias only
                flops = "N/A"  # FLOPs calculation not possible without more info

                total_params += params

                table_data.append([layer_type, layer_output_shape, params, flops])
        else:
            table_data.append(["Unknown Layer Type", layer_output_shape, "N/A", "N/A"])
    else:
        table_data.append(["Unknown Layer Type", layer_name, "N/A", "N/A"])

# Add a single entry for all convolutional layers
table_data.append(["Convolutional Layers (Total)", "N/A", conv_params, conv_flops])

# Print the table without warnings
headers = ["Layer Type", "Output Shape", "Parameters", "FLOPs"]
print(tabulate(table_data, headers=headers, tablefmt="plain"))
print(f"Total Parameters: {total_params}")
print(f"Total FLOPs: {total_flops}")


Layer Type                    Output Shape           Parameters    FLOPs
Unknown Layer Type            [  1 320 320   3]      N/A           N/A
Convolutional Layer           [3]                    4             N/A
Convolutional Layer           [3]                    4             N/A
Unknown Layer Type            [12804     4]          N/A           N/A
Unknown Layer Type            [4]                    N/A           N/A
Unknown Layer Type            [4]                    N/A           N/A
Unknown Layer Type            [4]                    N/A           N/A
Unknown Layer Type            [4]                    N/A           N/A
Convolutional Layer           [32]                   33            N/A
Convolutional Layer           [32]                   33            N/A
Unknown Layer Type            [96]                   N/A           N/A
Unknown Layer Type            [96]                   N/A           N/A
Unknown Layer Type            [144]                  N/A           N/A
Unkn

In [65]:
import tensorflow as tf

# Path to the directory containing the SavedModel
model_dir = "C:/UsersAI/Desktop/Tensorflow/custom_model_lite/saved_model/"

try:
    # Load the SavedModel
    loaded_model = tf.saved_model.load(model_dir)

    # Print available information
    print("SavedModel loaded successfully!")

    # Print signatures
    print("Signatures:")
    for signature_name in loaded_model.signatures:
        signature = loaded_model.signatures[signature_name]
        print(f"- {signature_name}:")
        print(f"  - Inputs: {signature.inputs}")
        print(f"  - Outputs: {signature.outputs}")

    # Print computational graph structure
    print("Computational Graph Structure:")
    print(loaded_model.graph.as_graph_def())

    # Print metadata (if available)
    if hasattr(loaded_model, "metadata"):
        print("Metadata:")
        print(loaded_model.metadata)

    # Access input and output tensors (if needed)
    # input_tensor = loaded_model.input_tensor
    # output_tensor = loaded_model.output_tensor

    # Access model parameters (if applicable)
    # model_parameters = loaded_model.trainable_variables

    # Perform specific tasks with the model (e.g., make predictions)

except OSError as e:
    print(f"Error loading SavedModel: {e}")
except Exception as e:
    print(f"An error occurred: {e}")


Error loading SavedModel: SavedModel file does not exist at: C:/UsersAI/Desktop/Tensorflow/custom_model_lite/saved_model/\{saved_model.pbtxt|saved_model.pb}


In [68]:
import numpy as np
import tensorflow as tf

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="C:/Users/AI/Desktop/Tensorflow/custom_model_lite/detect.tflite")
interpreter.allocate_tensors()

# Get input details
input_details = interpreter.get_input_details()

# Prepare example input data
input_data = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]], dtype=np.float32)

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get output details and results
output_details = interpreter.get_output_details()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)


ValueError: Cannot set tensor: Dimension mismatch. Got 2 but expected 4 for input 0.

In [89]:
import tensorflow as tf
from decimal import Decimal
from tabulate import tabulate

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="C:/Users/AI/Desktop/Tensorflow/custom_model_lite/detect.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Initialize dictionaries to store layer-wise parameters and FLOPs
layer_params = {}
layer_flops = {}

# Get details of all layers
for i, layer in enumerate(interpreter.get_tensor_details()):
    layer_name = layer['name']
    layer_output_shape = layer['shape']

    if len(layer_output_shape) > 0:  # Check if the layer has output shape
        layer_type = None
        
        # Check for specific keywords in the layer name to determine the type
        if 'conv' in layer_name.lower():
            if 'depthwise' in layer_name.lower():
                layer_type = 'Depthwise Convolution Layer'
            elif 'separable_conv2d' in layer_name.lower():
                layer_type = 'Separable Convolution Layer'
            else:
                layer_type = 'Convolution Layer'
        elif 'dense' in layer_name.lower() or 'fullyconnected' in layer_name.lower():
            layer_type = 'Fully Connected Layer'
        elif 'fusedbatchnormv3' in layer_name.lower():
            layer_type = 'Fused Batch Normalization Layer'
        elif 'readvariableop' in layer_name.lower():
            layer_type = 'Read Variable Operation'
        elif 'readvariableop1' in layer_name.lower():
            layer_type = 'Read Variable Operation 1'
        elif 'shape' in layer_name.lower():
            layer_type = 'Shape Layer'
        # Add more conditions for other types of layers as needed

        if layer_type:
            # Initialize layer-wise parameters and FLOPs
            if layer_type not in layer_params:
                layer_params[layer_type] = Decimal(0)
                layer_flops[layer_type] = Decimal(0)

            # Count parameters and FLOPs for convolutional layers
            if 'conv2d' in layer_name.lower() or 'depthwiseconv2d' in layer_name.lower() or 'separableconv2d' in layer_name.lower():
                if len(layer_output_shape) >= 4:  # Check if the shape has enough dimensions
                    filter_shape = layer_output_shape[1:3]
                    in_channels = layer_output_shape[3]
                    out_channels = layer_output_shape[0]
                    kernel_size = int(filter_shape[0]) * int(filter_shape[1])

                    # Convert numpy integers to regular integers before conversion to Decimal
                    params = Decimal(int(kernel_size * in_channels) * int(out_channels) + int(out_channels))  # Weight + bias
                    flops = Decimal(int(kernel_size * in_channels) * int(out_channels) * int(layer_output_shape[1]) * int(layer_output_shape[2]))  # MACs

                    layer_params[layer_type] += params
                    layer_flops[layer_type] += flops
                else:
                    layer_params[layer_type] += Decimal(0)
                    layer_flops[layer_type] += Decimal(0)
            else:
                # Assuming other layers (e.g., fully connected) have only bias parameters
                params = Decimal(int(layer_output_shape[0]) + 1)  # Bias only
                flops = Decimal(0)  # FLOPs calculation not relevant for non-convolutional layers

                layer_params[layer_type] += params

# Generate the final table for each layer type
table_data = []
for layer_type, params in layer_params.items():
    flops = layer_flops.get(layer_type, Decimal(0))
    table_data.append([layer_type, params, flops])

# Print the final table
headers = ["Layer Type", "Total Parameters", "Total FLOPs"]
print(tabulate(table_data, headers=headers, tablefmt="plain"))


Layer Type                         Total Parameters    Total FLOPs
Convolution Layer                       3.30622e+06    1.08417e+10
Shape Layer                            28              0
Depthwise Convolution Layer             1.26687e+07    1.27153e+11
Fused Batch Normalization Layer     14558              0
Separable Convolution Layer        759654              2.11387e+09


In [93]:
import tensorflow as tf
from decimal import Decimal
from tabulate import tabulate

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="C:/Users/AI/Desktop/Tensorflow/custom_model_lite/detect.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Initialize table data
table_data = []

# Get details of all layers
total_params = Decimal(0)  # Use Decimal for total_params to handle larger values
total_flops = Decimal(0)  # Use Decimal for total_flops to handle larger values

for i, layer in enumerate(interpreter.get_tensor_details()):
    layer_name = layer['name']
    layer_output_shape = layer['shape']

    if len(layer_output_shape) > 0:  # Check if the layer has output shape
        layer_type = None
        
        # Check for specific keywords in the layer name to determine the type
        if 'conv' in layer_name.lower():
            if 'depthwise' in layer_name.lower():
                layer_type = 'Depthwise Convolution Layer'
            elif 'separable_conv2d' in layer_name.lower():
                if 'separable_conv2d_1' in layer_name.lower():
                    layer_type = 'Separable Convolution Layer 1'
                elif 'separable_conv2d_2' in layer_name.lower():
                    layer_type = 'Separable Convolution Layer 2'
                elif 'separable_conv2d_3' in layer_name.lower():
                    layer_type = 'Separable Convolution Layer 3'
                else:
                    layer_type = 'Separable Convolution Layer'
            else:
                layer_type = 'Convolution Layer'
        elif 'dense' in layer_name.lower() or 'fullyconnected' in layer_name.lower():
            layer_type = 'Fully Connected Layer'
        elif 'fusedbatchnormv3' in layer_name.lower():
            layer_type = 'Fused Batch Normalization Layer'
        elif 'readvariableop' in layer_name.lower():
            layer_type = 'Read Variable Operation'
        elif 'readvariableop1' in layer_name.lower():
            layer_type = 'Read Variable Operation 1'
        elif 'shape' in layer_name.lower():
            layer_type = 'Shape Layer'
        # Add more conditions for other types of layers as needed

        if layer_type:
            # Count parameters and FLOPs for convolutional layers
            if 'conv2d' in layer_name.lower() or 'depthwiseconv2d' in layer_name.lower() or 'separableconv2d' in layer_name.lower():
                if len(layer_output_shape) >= 4:  # Check if the shape has enough dimensions
                    filter_shape = layer_output_shape[1:3]
                    in_channels = layer_output_shape[3]
                    out_channels = layer_output_shape[0]
                    kernel_size = int(filter_shape[0]) * int(filter_shape[1])

                    # Convert numpy integers to regular integers before conversion to Decimal
                    params = Decimal(int(kernel_size * in_channels) * int(out_channels) + int(out_channels))  # Weight + bias
                    flops = Decimal(int(kernel_size * in_channels) * int(out_channels) * int(layer_output_shape[1]) * int(layer_output_shape[2]))  # MACs

                    total_params += params
                    total_flops += flops
                    
                    table_data.append([layer_type, layer_output_shape, params, flops, layer['shape']])
                else:
                    table_data.append([layer_type, layer_output_shape, "N/A", "N/A", layer['shape']])
            else:
                # Assuming other layers (e.g., fully connected) have only bias parameters
                params = Decimal(int(layer_output_shape[0]) + 1)  # Bias only
                flops = "N/A"  # FLOPs calculation not possible without more info

                total_params += params

                table_data.append([layer_type, layer_output_shape, params, flops, layer['shape']])
        else:
            table_data.append(["Unknown Layer Type", layer_output_shape, "N/A", "N/A", layer['shape']])
    else:
        table_data.append(["Unknown Layer Type", layer_name, "N/A", "N/A", "N/A"])

# Print the table without warnings
headers = ["Layer Type", "Output Shape", "Parameters", "FLOPs", "Layer Shape"]
print(tabulate(table_data, headers=headers, tablefmt="plain"))
print(f"Total Parameters: {total_params}")
print(f"Total FLOPs: {total_flops}")


Layer Type                       Output Shape           Parameters    FLOPs        Layer Shape
Unknown Layer Type               [  1 320 320   3]      N/A           N/A          [  1 320 320   3]
Convolution Layer                [3]                    4             N/A          [3]
Convolution Layer                [3]                    4             N/A          [3]
Unknown Layer Type               [12804     4]          N/A           N/A          [12804     4]
Shape Layer                      [4]                    5             N/A          [4]
Shape Layer                      [4]                    5             N/A          [4]
Shape Layer                      [4]                    5             N/A          [4]
Shape Layer                      [4]                    5             N/A          [4]
Convolution Layer                [32]                   33            N/A          [32]
Depthwise Convolution Layer      [32]                   33            N/A          [32]
Fused Bat

In [98]:
import tensorflow as tf
from decimal import Decimal
from tabulate import tabulate

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="C:/Users/AI/Desktop/Tensorflow/custom_model_lite/detect.tflite")
interpreter.allocate_tensors()

# Initialize dictionaries to store layer-wise parameters and FLOPs
layer_params = {}
layer_flops = {}

# Get details of all layers
for i, layer in enumerate(interpreter.get_tensor_details()):
    layer_name = layer['name']
    layer_output_shape = layer['shape']

    if len(layer_output_shape) > 0:  # Check if the layer has output shape
        layer_type = None
        
        # Check for specific keywords in the layer name to determine the type
        if 'conv' in layer_name.lower():
            if 'depthwise' in layer_name.lower():
                layer_type = 'Depthwise Convolution Layer'
            elif 'separable_conv2d' in layer_name.lower():
                if 'separable_conv2d_1' in layer_name.lower():
                    layer_type = 'Separable Convolution Layer 1'
                elif 'separable_conv2d_2' in layer_name.lower():
                    layer_type = 'Separable Convolution Layer 2'
                elif 'separable_conv2d_3' in layer_name.lower():
                    layer_type = 'Separable Convolution Layer 3'
                else:
                    layer_type = 'Separable Convolution Layer'
            else:
                layer_type = 'Convolution Layer'
        elif 'dense' in layer_name.lower() or 'fullyconnected' in layer_name.lower():
            layer_type = 'Fully Connected Layer'
        elif 'fusedbatchnormv3' in layer_name.lower():
            layer_type = 'Fused Batch Normalization Layer'
        elif 'readvariableop' in layer_name.lower():
            layer_type = 'Read Variable Operation'
        elif 'readvariableop1' in layer_name.lower():
            layer_type = 'Read Variable Operation 1'
        elif 'shape' in layer_name.lower():
            layer_type = 'Shape Layer'
        # Add more conditions for other types of layers as needed

        if layer_type:
            # Initialize layer-wise parameters and FLOPs
            if layer_type not in layer_params:
                layer_params[layer_type] = Decimal(0)
                layer_flops[layer_type] = Decimal(0)

            # Count parameters and FLOPs for convolutional layers
            if 'conv2d' in layer_name.lower() or 'depthwiseconv2d' in layer_name.lower() or 'separableconv2d' in layer_name.lower():
                if len(layer_output_shape) >= 4:  # Check if the shape has enough dimensions
                    filter_shape = layer_output_shape[1:3]
                    in_channels = layer_output_shape[3]
                    out_channels = layer_output_shape[0]
                    kernel_size = int(filter_shape[0]) * int(filter_shape[1])

                    # Convert numpy integers to regular integers before conversion to Decimal
                    params = Decimal(int(kernel_size * in_channels) * int(out_channels) + int(out_channels))  # Weight + bias
                    flops = Decimal(int(kernel_size * in_channels) * int(out_channels) * int(layer_output_shape[1]) * int(layer_output_shape[2]))  # MACs

                    layer_params[layer_type] += params
                    layer_flops[layer_type] += flops
                else:
                    layer_params[layer_type] += Decimal(0)
                    layer_flops[layer_type] += Decimal(0)
            else:
                # Assuming other layers (e.g., fully connected) have only bias parameters
                params = Decimal(int(layer_output_shape[0]) + 1)  # Bias only
                flops = Decimal(0)  # FLOPs calculation not relevant for non-convolutional layers

                layer_params[layer_type] += params

# Generate the final table for each layer type
table_data = []
for layer_type, params in layer_params.items():
    flops = layer_flops.get(layer_type, Decimal(0))
    table_data.append([layer_type, params, flops])

# Calculate overall totals
overall_params = sum(layer_params.values())
overall_flops = sum(layer_flops.values())

# Add the overall totals to the table
table_data.append(["Overall Total", overall_params, overall_flops])

# Print the final table
headers = ["Layer Type", "Total Parameters", "Total FLOPs"]
print(tabulate(table_data, headers=headers, tablefmt="plain"))



Layer Type                         Total Parameters       Total FLOPs
Convolution Layer                       3.30622e+06       1.08417e+10
Shape Layer                            28                 0
Depthwise Convolution Layer             1.26687e+07       1.27153e+11
Fused Batch Normalization Layer     14558                 0
Separable Convolution Layer        533154                 2.10446e+09
Separable Convolution Layer 1       87650                 8.70554e+06
Separable Convolution Layer 2       71450            605536
Separable Convolution Layer 3       67400             99286
Overall Total                           1.67491e+07       1.40109e+11


In [42]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras_flops import get_flops

def build_model(input_shape=(32, 32, 3), num_classes=10):
    inp = Input(input_shape)
    x = Conv2D(32, kernel_size=(3, 3), activation="relu")(inp)
    x = Conv2D(64, (3, 3), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.5)(x)
    out = Dense(num_classes, activation="softmax")(x)
    model = Model(inp, out)
    return model

def calculate_flops(model, batch_size=1):
    flops = get_flops(model, batch_size=batch_size)
    return flops / 10 ** 9  # Convert to GFLOPS

# Usage example:
input_shape = (32, 32, 3)
num_classes = 10
batch_size = 1

model = build_model(input_shape, num_classes)
flops = calculate_flops(model, batch_size)
print(f"FLOPS: {flops:.03} GFLOPS")


FLOPS: 0.0338 GFLOPS


In [38]:
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

def build_model(input_shape=(32, 32, 3), num_classes=10):
    inp = Input(input_shape)
    x = Conv2D(32, kernel_size=(3, 3), activation="relu")(inp)
    x = Conv2D(64, (3, 3), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.5)(x)
    out = Dense(num_classes, activation="softmax")(x)
    model = Model(inp, out)
    return model

def calculate_parameters(model):
    total_params = model.count_params()
    return total_params

def estimate_flops(layer, input_shape=(32, 32, 3)):
    input_size = input_shape[0] * input_shape[1] * input_shape[2]
    flops = 0  # Initialize flops with a default value
    if isinstance(layer, Conv2D):
        output_size = layer.output_shape[1] * layer.output_shape[2] * layer.output_shape[3]
        flops = (2 * output_size * layer.kernel_size[0] * layer.kernel_size[1] * layer.filters) / layer.strides[0]
    elif isinstance(layer, Dense):
        output_size = layer.units
        flops = (2 * input_size * output_size)  # FLOPs for Dense layers
    return flops

# Usage example:
input_shape = (32, 32, 3)
num_classes = 10
batch_size = 1

model = build_model(input_shape, num_classes)

# Calculate total parameters
total_params = calculate_parameters(model)
print(f"Total Parameters: {total_params}")

# Estimate FLOPs for each layer
layer_names = [layer.name for layer in model.layers[1:]]  # Exclude the input layer
for idx, layer in enumerate(model.layers[1:]):  # Exclude the input layer
    layer_flops = estimate_flops(layer, input_shape)
    layer_params = layer.count_params()
    print(f"Layer: {layer_names[idx]}")
    print(f"Estimated FLOPs: {layer_flops} FLOPs")  # Removed precision specifier
    print(f"Parameters: {layer_params}")
    print()


Total Parameters: 1626442
Layer: conv2d_56
Estimated FLOPs: 16588800.0 FLOPs
Parameters: 896

Layer: conv2d_57
Estimated FLOPs: 57802752.0 FLOPs
Parameters: 18496

Layer: max_pooling2d_28
Estimated FLOPs: 0 FLOPs
Parameters: 0

Layer: dropout_56
Estimated FLOPs: 0 FLOPs
Parameters: 0

Layer: flatten_28
Estimated FLOPs: 0 FLOPs
Parameters: 0

Layer: dense_56
Estimated FLOPs: 786432 FLOPs
Parameters: 1605760

Layer: dropout_57
Estimated FLOPs: 0 FLOPs
Parameters: 0

Layer: dense_57
Estimated FLOPs: 61440 FLOPs
Parameters: 1290



In [41]:
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

def build_model(input_shape=(32, 32, 3), num_classes=10):
    inp = Input(input_shape)
    x = Conv2D(32, kernel_size=(3, 3), activation="relu")(inp)
    x = Conv2D(64, (3, 3), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.5)(x)
    out = Dense(num_classes, activation="softmax")(x)
    model = Model(inp, out)
    return model

def calculate_total_parameters(model):
    return model.count_params()

def estimate_total_flops(model, input_shape=(32, 32, 3)):
    total_flops = 0
    for layer in model.layers[1:]:  # Exclude the input layer
        total_flops += estimate_flops(layer, input_shape)
    return total_flops

def estimate_flops(layer, input_shape=(32, 32, 3)):
    input_size = input_shape[0] * input_shape[1] * input_shape[2]
    flops = 0  # Initialize flops with a default value
    if isinstance(layer, Conv2D):
        output_size = layer.output_shape[1] * layer.output_shape[2] * layer.output_shape[3]
        flops = (2 * output_size * layer.kernel_size[0] * layer.kernel_size[1] * layer.filters) / layer.strides[0]
    elif isinstance(layer, Dense):
        output_size = layer.units
        flops = (2 * input_size * output_size)  # FLOPs for Dense layers
    return flops

# Usage example:
input_shape = (32, 32, 3)
num_classes = 10
batch_size = 1

model = build_model(input_shape, num_classes)

# Calculate total parameters
total_params = calculate_total_parameters(model)
print(f"Total Parameters: {total_params}")

# Calculate total FLOPs
total_flops = estimate_total_flops(model, input_shape)
print(f"Total FLOPs: {total_flops} FLOPs")



Total Parameters: 1626442
Total FLOPs: 75239424.0 FLOPs


In [44]:
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

def build_model(input_shape=(32, 32, 3), num_classes=10):
    inp = Input(input_shape)
    x = Conv2D(32, kernel_size=(3, 3), activation="relu")(inp)
    x = Conv2D(64, (3, 3), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.5)(x)
    out = Dense(num_classes, activation="softmax")(x)
    model = Model(inp, out)
    return model

def calculate_parameters(model):
    total_params = model.count_params()
    return total_params

def estimate_flops(layer, input_shape=(32, 32, 3)):
    input_size = input_shape[0] * input_shape[1] * input_shape[2]
    flops = 0  # Initialize flops with a default value
    if isinstance(layer, Conv2D):
        output_size = layer.output_shape[1] * layer.output_shape[2] * layer.output_shape[3]
        flops = (2 * output_size * layer.kernel_size[0] * layer.kernel_size[1] * layer.filters) / layer.strides[0]
    elif isinstance(layer, Dense):
        output_size = layer.units
        flops = (2 * input_size * output_size)  # FLOPs for Dense layers
    return flops

# Usage example:
input_shape = (32, 32, 3)
num_classes = 10
batch_size = 1

model = build_model(input_shape, num_classes)

# Calculate total parameters
total_params = calculate_parameters(model)

# Estimate FLOPs for each layer
total_flops = 0
layer_names = [layer.name for layer in model.layers[1:]]  # Exclude the input layer
for idx, layer in enumerate(model.layers[1:]):  # Exclude the input layer
    layer_flops = estimate_flops(layer, input_shape)
    total_flops += layer_flops
    layer_params = layer.count_params()
    print(f"Layer: {layer_names[idx]}")
    print(f"Estimated FLOPs: {layer_flops} FLOPs")
    print(f"Parameters: {layer_params}")
    print()

print(f"Total FLOPs: {total_flops} FLOPs")
print(f"Total Parameters: {total_params}")


Layer: conv2d_68
Estimated FLOPs: 16588800.0 FLOPs
Parameters: 896

Layer: conv2d_69
Estimated FLOPs: 57802752.0 FLOPs
Parameters: 18496

Layer: max_pooling2d_34
Estimated FLOPs: 0 FLOPs
Parameters: 0

Layer: dropout_68
Estimated FLOPs: 0 FLOPs
Parameters: 0

Layer: flatten_34
Estimated FLOPs: 0 FLOPs
Parameters: 0

Layer: dense_68
Estimated FLOPs: 786432 FLOPs
Parameters: 1605760

Layer: dropout_69
Estimated FLOPs: 0 FLOPs
Parameters: 0

Layer: dense_69
Estimated FLOPs: 61440 FLOPs
Parameters: 1290

Total FLOPs: 75239424.0 FLOPs
Total Parameters: 1626442


In [61]:
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

def build_model(input_shape=(32, 32, 3), num_classes=10):
    inp = Input(input_shape)
    x = Conv2D(32, kernel_size=(3, 3), activation="relu")(inp)
    x = Conv2D(64, (3, 3), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.5)(x)
    out = Dense(num_classes, activation="softmax")(x)
    model = Model(inp, out)
    return model

def calculate_parameters(model):
    total_params = model.count_params()
    return total_params

def estimate_flops(layer, input_shape=(32, 32, 3)):
    input_size = input_shape[0] * input_shape[1] * input_shape[2]
    flops = 0  # Initialize flops with a default value
    if isinstance(layer, Conv2D):
        # Ensure padding values are integers
        padding_0 = int(layer.padding[0]) if isinstance(layer.padding[0], int) else 0
        padding_1 = int(layer.padding[1]) if isinstance(layer.padding[1], int) else 0
        # Calculate output size based on input shape and Conv2D parameters
        output_height = ((input_shape[0] - layer.kernel_size[0] + 2 * padding_0) // layer.strides[0]) + 1
        output_width = ((input_shape[1] - layer.kernel_size[1] + 2 * padding_1) // layer.strides[1]) + 1
        output_channels = layer.filters
        # Calculate FLOPs for Conv2D layer
        flops = output_height * output_width * layer.kernel_size[0] * layer.kernel_size[1] * input_shape[2] * output_channels
    elif isinstance(layer, Dense):
        output_size = layer.units
        # Calculate FLOPs for Dense layer
        flops = 2 * input_size * output_size + output_size  # Include FLOPs for biases
    return flops

# Usage example:
input_shape = (32, 32, 3)
num_classes = 10
batch_size = 1

model = build_model(input_shape, num_classes)

# Calculate total parameters
total_params = calculate_parameters(model)

# Estimate FLOPs for each layer
total_flops = 0
layer_names = [layer.name for layer in model.layers[1:]]  # Exclude the input layer
for idx, layer in enumerate(model.layers[1:]):  # Exclude the input layer
    layer_flops = estimate_flops(layer, input_shape)
    total_flops += layer_flops
    layer_params = layer.count_params()
    print(f"Layer: {layer_names[idx]}")
    print(f"Estimated FLOPs: {layer_flops} FLOPs")
    print(f"Parameters: {layer_params}")
    print()

print(f"Total FLOPs: {total_flops} FLOPs")
print(f"Total Parameters: {total_params}")


Layer: conv2d_102
Estimated FLOPs: 777600 FLOPs
Parameters: 896

Layer: conv2d_103
Estimated FLOPs: 1555200 FLOPs
Parameters: 18496

Layer: max_pooling2d_51
Estimated FLOPs: 0 FLOPs
Parameters: 0

Layer: dropout_102
Estimated FLOPs: 0 FLOPs
Parameters: 0

Layer: flatten_51
Estimated FLOPs: 0 FLOPs
Parameters: 0

Layer: dense_102
Estimated FLOPs: 786560 FLOPs
Parameters: 1605760

Layer: dropout_103
Estimated FLOPs: 0 FLOPs
Parameters: 0

Layer: dense_103
Estimated FLOPs: 61450 FLOPs
Parameters: 1290

Total FLOPs: 3180810 FLOPs
Total Parameters: 1626442
